In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import sys
import os

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.dpi'] = 300
# plt.rcParams['figure.figsize'] = (10,4)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['legend.fontsize'] = 7
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

In [ ]:
name = 'OakMixed'
run = 2

area = {
    "Naches": 2609453362.5281982,# mid naches 709539162.2819214,
    "UpperWenas": 251671142.33624268,
    "ame350": 188350967.47338867,
    "American": 188350967.47338867,
    "Mckenzie": 2731395925.1401367,
    "UpperYakima": 2609453362.5281982,
    "Satus" : 1300000000,
    "Oak": 74151215.62976074,
    "OakMixed": 74151215.62976074,
}# hard coded to avoid reading mesh file

AREA = area[name]

MOL = 55000

if run == 0:
    run = f'run{run}-spinup_steadystate'
elif run == 1:
    run = f'run{run}-spinup_cyclic'
elif run == 2:
    run = f'run{run}-transient'

path = os.environ.get('SCRATCH')+f'/{name}/{run}/{name}-?/'
path

In [ ]:
if 'run1' in path or 'run2' in path:
    # fl = sorted(glob.glob(path+'water_balance_computational_domain.csv'), key=lambda y: int(y.split('-')[2].split('/')[0]))
    fl = sorted(glob.glob(path+'water_balance_computational_domain.csv'))
    print(fl)
    print('#'*52)
    break_pts = []
    for i, f in enumerate(fl):
        if f == fl[0]:
            df = pd.read_csv(f, comment='#')
            df.iloc[0,1:] = df.iloc[1,1:]
            print(f, 'start =', df.iloc[0,0], 'end =', df.iloc[-1,0])
        else:
            break_pts.append(len(df))
            _df = pd.read_csv(f, comment='#')
            if _df.iloc[0,0] != df.iloc[-1,0] + 1:
                while _df.iloc[0,0] != df.iloc[-1,0] + 1:
                    _df = _df.iloc[1:, :]
            assert(_df.iloc[0,0] == df.iloc[-1,0] + 1)
            df = pd.concat([df, _df])
            print(f, 'start =', _df.iloc[0,0], 'end =', _df.iloc[-1,0])
    print('#'*52)
    df.reset_index(drop=True, inplace=True)
    for i in range(3):
        df[df.columns[-4]+' '] = df.iloc[:,-4]
        df.drop(columns=[df.columns[-5]], inplace=True)
    for i in range(2):
        df[df.columns[4]+' '] = df.iloc[:,4]
        df.drop(columns=[df.columns[4]], inplace=True)
    df.rename(columns={"time [d]": "time"}, inplace=True)
    for i in df.columns:
        df.rename(columns={i: i.replace('] ', ']')}, inplace=True)
    print(df.columns)
    
    t = df.iloc[:,0].to_numpy()
    t00 = t[0]
    t0 = '1980-01-01'
    df.iloc[:,0] = pd.to_datetime(t, unit='D', origin=pd.Timestamp(t0))

    if 'run2' in path:
        t0 = t0[:-2] + str(int(int(t0[-2:])+t[0]//(365*4))).zfill(2)
        df.iloc[:,0] = pd.to_datetime(t, unit='D', origin=pd.Timestamp(t0))
        print('#'*52)
        print('t0 changed to', t0, 'because', str(int(t[0]))+'//(365*4) =', int(t[0]//(365*4)))
        print('BEFORE LEAP DAYS REMOVAL:')
        print('  start =', df.iloc[0,0])
        print('  end =', df.iloc[-1,0])
        print('  total =', len(df.iloc[:,0]), 'days')
        print()
        print('  start =', t[0])
        print('  end =', t[-1])
        print('  total =', len(t), 'days')
        print()
        print('LEAP DAYS LIST:')
        lp = []
        for i,j in enumerate(df.iloc[:,0]):
            if str(j)[5:10] == '02-29':
                lp.append(i)
                print('  '+str(j)[:10], 'index =', i)
        print('  total =', len(lp), 'days')
        print()
        t = np.delete(t, lp)
        print(len(lp), 'LEAP DAYS DROPPED')
        print('  start =', t[0])
        print('  end =', t[-1])
        print('  total =', len(t), 'days')
        print()
        print('APPEND DAYS TO LAST')
        print(t[-1], np.arange(t[-1]+1, t[-1]+len(lp)+1))
        t = np.concatenate([t, np.arange(t[-1]+1,t[-1]+len(lp)+1)], axis=None)
        print('  new start =', t[0])
        print('  new end =', t[-1])
        print('  new total =', len(t), 'days')
        print()
        t = pd.to_datetime(t, unit='D', origin=pd.Timestamp(t0))
        print('AFTER LEAP DAYS REMOVAL AND APPENDING:')
        print('  start =', t[0])
        print('  end =', t[-1])
        print('  total =', len(t), 'days')
        print('#'*52)
        df.iloc[:,0] = t

    print('#'*52)
    print()
    
    axs = df.plot(x=df.columns[0], subplots=True, layout=(6,3), figsize=(12, 6), zorder=1, legend=False, fontsize=6)
    for i, ax in enumerate(axs.flat):
        ax.grid()
        ax.set_title(df.columns[i+1], fontsize=8)
        for break_pt in break_pts:
            ax.scatter(x=df.iloc[break_pt,0], y=df.iloc[break_pt,i+1], s=3, c='k', zorder=2)
    plt.tight_layout()
    plt.show()

    print()
    df.iloc[:,1] = df.iloc[:,1]/MOL/AREA
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 5))
    ax = ax1.twinx()
    ax1.set_ylabel('outgoing flux [m/d]', fontsize=10)
    ax.set_ylabel('incoming flux [m/d]', fontsize=10)
    ax1.plot(df.iloc[:,0], df.iloc[:,1], 'tab:red', label='watershed boundary discharge')
    ax1.plot(df.iloc[:,0], df.iloc[:,4]+df.iloc[:,5]+df.iloc[:,6]+df.iloc[:,9], color='tab:green', label='total ET')
    for break_pt in break_pts:
        ax1.scatter(df.iloc[break_pt,0], df.iloc[break_pt,1], s=3, c='k', zorder=2)
    ax1.set_ylim(-np.max(df.iloc[:,1])/10, np.max(df.iloc[:,1])*2)
    ax1.legend(loc='center left')
    ax.bar(df.iloc[:,0], df.iloc[:,-2], width=1, alpha=0.6, label='rain', color='tab:blue')
    ax.bar(df.iloc[:,0], df.iloc[:,-1], width=1, alpha=0.6, label='snow', color='darkgrey')
    ax.bar(df.iloc[:,0], df.iloc[:,7], width=1, alpha=0.6, label='snowmelt', color='tab:orange')
    ax.set_ylim(-np.max(df.iloc[:,1])/10, np.max(df.iloc[:,-2])*2)
    ax.invert_yaxis()
    ax.legend(loc='center right')
    
    ax2.set_ylabel('cum flux [m]', fontsize=10)
    ax2.plot(df.iloc[:,0], np.cumsum(df.iloc[:,1]), color='tab:red', label='cum_watershed boundary discharge')
    ax2.plot(df.iloc[:,0], np.cumsum(df.iloc[:,4]+df.iloc[:,5]+df.iloc[:,6]+df.iloc[:,9]), color='tab:green', label='cum_total ET')
    ax2.plot(df.iloc[:,0], np.cumsum(df.iloc[:,-2]), label='cum_rain', color='tab:blue')
    ax2.plot(df.iloc[:,0], np.cumsum(df.iloc[:,-1]), label='cum_snow', color='darkgrey')
    ax2.plot(df.iloc[:,0], np.cumsum(df.iloc[:,7]), label='cum_snowmelt', color='tab:orange')
    ax2.plot(df.iloc[:,0], np.cumsum(df.iloc[:,-1]+df.iloc[:,-2]-(df.iloc[:,4]+df.iloc[:,5]+df.iloc[:,6]+df.iloc[:,9])-df.iloc[:,1]), label='cum_(Pr+Ps-ET-Q)', color='cyan', linestyle='--')
    mass = (df.iloc[:,10]+df.iloc[:,11]+df.iloc[:,12]+df.iloc[:,13])/MOL/AREA
    ax2.plot(df.iloc[:,0], mass - mass[0], label='water mass change', color='peru')
    error = mass - mass[0] - np.cumsum(df.iloc[:,-1]+df.iloc[:,-2]-(df.iloc[:,4]+df.iloc[:,5]+df.iloc[:,6]+df.iloc[:,9])-df.iloc[:,1])
    ax2.plot(df.iloc[:,0], error, label='water mass error', color='black', linestyle=':')
    ax2.legend(loc='upper left', ncol=2)
    max_error = np.max(error) if np.max(error) == np.max(np.abs(error)) else -np.max(np.abs(error))
    ax2.set_title('max error = '+str(np.round(max_error, 4))+' m', fontsize=10)
    
    ax1.grid()
    ax.grid()
    ax2.grid()
    
    plt.tight_layout()
    plt.show()

elif 'run0' in path:
    # fl = sorted(glob.glob(path+'water_balance.dat'), key=lambda y: int(y.split('-')[2].split('/')[0]))
    fl = glob.glob(path+'water_balance.dat')
    print(fl)

    break_pts = []
    for f in fl:
        if f == fl[0]:
            df = pd.read_csv(f, comment='#')
        else:
            break_pts.append(len(df))
            _df = pd.read_csv(f, comment='#')
            df = pd.concat([df, _df])
    df.reset_index(drop=True, inplace=True)
    df.iloc[:,0] /= 365
    df.rename(columns={"time [d]": "time [yr]"}, inplace=True)

    axs = df.plot(x=df.columns[0], subplots=True, layout=(2,2), figsize=(6,6), logy=False)
    for i, ax in enumerate(axs.flat):
        ax.grid(which='both', color='silver', linewidth=0.6)
        for break_pt in break_pts:
            ax.scatter(x=df.iloc[break_pt,0], y=df.iloc[break_pt,i+1], s=3, c='k', zorder=2)
    plt.tight_layout()
    plt.show()
    print()
    df.iloc[:,-1] = df.iloc[:,0]*365/10
    df.rename(columns={df.columns[-1]: 'time step [d]'}, inplace=True)
    t = df.iloc[:,0]
    df = np.abs(df.diff())
    df.iloc[:,0] = t
    for i in range(1,4):
        df.iloc[:,i] /= df.iloc[:,-1]

    axs = df.plot(x=df.columns[0], subplots=True, layout=(2,2), figsize=(6,6), logy=True)
    for i, ax in enumerate(axs.flat):
        ax.grid(which='both', color='silver', linewidth=0.6)
        for break_pt in break_pts:
            ax.scatter(x=df.iloc[break_pt,0], y=df.iloc[break_pt,i+1], s=3, c='k', zorder=2)
    plt.tight_layout()
    plt.show()

else:
    print('not in right dir')
    print('expecting run0, run1, run2')